In [5]:
import re
import pymupdf
import spacy

# All task  related to Job desc

In [6]:
#extract jobdescription
# Job_desc
with open ("job_desc.txt",'r') as f:
    Job_desc = f.read()

Cleaning Jobdesc

In [7]:
# cleaning --> to lower case convert regex matching
Job_desc = Job_desc.lower()
Job_desc = re.sub('[^a-zA-Z]',' ',Job_desc)

In [8]:
#nlp -->lemmitization and stop word removing
nlp = spacy.load("en_core_web_sm")
doc_jobDesc = nlp(Job_desc)
filtered_token = [token.lemma_ for token in doc_jobDesc if not token.is_stop]
corpus_jdsc=" ".join(filtered_token)
print(corpus_jdsc)

position   software developer   stack    backend   location   remote    kolkata   india employment type   internship    time role seek motivated skilled software developer strong expertise javascript typescript   c     python   couple hand experience backend system   web development   problem solve data structure    algorithm   ideal candidate passionate build scalable application   work modern framework   solve real world technical challenge   key responsibility design   develop   maintain stack application js   react js   express js   mongodb mysql   implement rest apis graphql service   ensure performance security   work machine learning powered feature recommendation system ai assist workflow   collaborate team analyze requirement create system design   uml   diagram   etc     apply strong problem solve algorithmic thinking optimize backend performance   develop   test   deploy application docker   git   ci cd workflow   ensure seamless user experience authentication system   oauth

# All task related to Resume pdf

In [9]:
doc = pymupdf.open("Avishek's resume.pdf")

In [10]:
#extract text from the pdf
out = open("output.txt","wb")
for page in doc:
    text = page.get_text().encode("utf8")
    out.write(text)
    out.write(bytes((12,)))
out.close()

In [11]:

resume_text=""
with open("output.txt","r",encoding="utf8") as f:
    resume_text = f.read()
candidate_email = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", resume_text)
print(candidate_email)

['patraavishe881@gmail.com']


Cleanig Resume text

In [12]:
#cleaning Resume text
resume_text = re.sub('[^a-zA-Z]',' ',resume_text)
resume_text = resume_text.strip()
resume_text = resume_text.lower()
resume_text = re.sub(r"[0-9]","",resume_text)
resume_text = re.sub(r"(linkedin|github)","",resume_text)
resume_text = re.sub(r"link","",resume_text)
resume_text = re.sub(r"(:|-)","",resume_text)
candidate_email = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", resume_text)
resume_text = re.sub(r"\S+@\S+","",resume_text)
resume_text = re.sub(r"(email|gmail|mail)","",resume_text)
resume_text = re.sub(r"(contact|mobile|phone)","",resume_text)
resume_text = re.sub(r"http?s://\S+","",resume_text)
resume_text = re.sub(r"www\.\S+","",resume_text)
resume_text = re.sub(r'[•*\-+]', '', resume_text)

In [13]:
#remove stop words
doc_resume = nlp(resume_text)
token_resume = [token.lemma_ for token in doc_resume if not token.is_stop]
corpus_resume = ' '.join(token_resume)
print(corpus_resume)

'avishek patra                                                                                                                                                       patraavishe      com                                                                                                                                                                                            education   ramakrishnan mission vivekanada                                              july                                                vivekanada centenary college                                                                rahara   kolkata   india   bachelor s computer science    gpa                                                                                                                                              skill summary     language                                  c     javascript typescript python     framework                            js   express js   react js   langchain     tool    tech

# Skill matchin by spacy rule based matching

In [90]:
import pandas as pd
import numpy as np
from spacy.matcher import Matcher
import spacy

In [91]:
skill_csv = pd.read_csv('skill_list.csv')
skill_array = np.array(skill_csv['skill'])

In [99]:
#spacy rule based matching
nlp_match = spacy.load("en_core_web_sm")
doc = nlp_match(corpus_jdsc)
matcher = Matcher(nlp_match.vocab)
#first have to match between skill list and jobdesc
for skill in skill_array:
    skill = skill.lower();
    pattern = [{"LOWER": skill}]  
    matcher.add(skill, [pattern])
matches = matcher(doc)


In [102]:
jobdesc_skill = set()
for match_id, start, end in matches:
    string_id = nlp_match.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    jobdesc_skill.add(string_id)
print(jobdesc_skill)  

{'mysql', 'typescript', 'postman', 'c', 'graphql', 'python', 'git', 'react', 'express', 'docker', 'langchain', 'mongodb', 'javascript'}


# TF-IDF Vectorizer

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [48]:
tf_idf = TfidfVectorizer()
vectors = tf_idf.fit_transform([corpus_resume,corpus_jdsc])

In [49]:
#finding cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(vectors[0],vectors[1])
similarity_score

array([[0.37576434]])